### Encontrando Conectores-Chave

<p>Contexto: Meu primeiro dia de trabalho na DataSciencester e o vice-presidente de Rede (networking) está cheio de perguntas sobre o usuário.</p>

<p>Ele quer que eu identifique quem são os "conectores-chave" entre os cientistas de dados. Para isso, me fornece uma parte de toda rede da DataSciencester.</p>

- Como se parece essa parte dos Dados? 
Consiste em uma lista de usuários, cada um representado por um <code>dict</code> que contém um <code>id (number)</code> para cada usuário(a) e um name. 

LET'S CODE!!!

In [12]:
users = [
    {"id": 0, "name": "Hero"},
    {"id": 1, "name": "Dunn"},
    {"id": 2, "name": "Sue"},
    {"id": 3, "name": "Chi"},
    {"id": 4, "name": "Thor"},
    {"id": 5, "name": "Clive"},
    {"id": 6, "name": "Hicks"},
    {"id": 7, "name": "Devin"},
    {"id": 8, "name": "Kate"},
    {"id": 9, "name": "Klein"},
]

## Dados "amigávéis" - representados por uma lista de pares IDs

friendships = [ (0,1),(0,2),(1,2),(1,3),(2,3),(3,4),
                (4,5),(5,6),(5,7),(6,8),(7,8),(8,9)
            ]

## Nesse cenário a tupla (0,1) indica que os cientistas de dados 
# com o id 0 e o id 1 são amigos.

<img src="A rede DataSciencester.png" alt="rede DataSciencester - Livro"/>

A rede DataSciencester

In [14]:
## Adicionado uma lista de amigos para cada usuário. 

# 1 - Configurando a propriedade 'friends' para cada usuário em uma lista vazia

for user in users: user["friends"] = []

# 2 - Povoando a lista com os dados de 'friendships'

for i, j in friendships:
    # Funciona porque users[i] é o usuário cujo o id é i
    users[i]["friends"].append(users[j]) # Adiciona como amigo de j
    users[j]["friends"].append(users[i]) # Adiciona como amigo de i

### Qual é o número médio de conexões? 

In [16]:
## 1 - Encontramos o número total de conexões, resumindo os tamanhos de todas as listas de friends 
# Quantos amigos o usuário tem? 
def number_of_friends(user):
    return len(user["friends"]) # Tamanho da lista friend_ids
total_connections = sum(number_of_friends(user) for user in users)

print(total_connections)


24


In [17]:
## 2 - Apenas divimos pelo número de usuários 
# from_future_imports division --- Divisão inteira está incompleta
num_users = len(users)
avg_connections = total_connections / num_users

print(avg_connections)

2.4


Também posso encontrar as pessoas mais conectadas - são as que possuem o maior número de amigos. 

Como não há muitos usuários, posso ordená-los de "muito amigos" para "menos amigos"

In [25]:
## Criando uma lista (user_id, number_of_friends)

num_friends_by_id = [(user["id"], number_of_friends(user)) for user in users]

sorted_num_friends_by_id = sorted(num_friends_by_id, 
       key=lambda user_id_num_friends : user_id_num_friends,
       reverse=True)

print(sorted_num_friends_by_id)

# 'user_id_num_friends' é a tupla que representa cada elemento 
# em 'num_friends_by_id'

[(9, 1), (8, 3), (7, 2), (6, 2), (5, 3), (4, 2), (3, 3), (2, 3), (1, 3), (0, 2)]


### Cientistas de Dados que você talvez conheça

Para estimular conexões entre os seus membros, tenho a ideia de desenvolver sugestões de 'Cientistas de dados que você talvez conheça'.
Como primeiro instinto surge sigerir um usuário que possa conhecer amigos de amigos. 

Fica mais fácil de computar: para cada amigo de um usuário, itera sobre os amigos daquela pessoa, e coleta todos os resultados. 

In [34]:
def friends_of_ids_bad(user):
    return [foaf["id"] # 'foaf é abreviação de friends of a friend'
            for friend in user["friends"] # para cada amigo de usuário
            for foaf in friend["friends"] # pega cada _their_friends
            ]

print([friend["id"] for friend in users[0]["friends"]])
print([friend["id"] for friend in users[1]["friends"]])
print([friend["id"] for friend in users[2]["friends"]])

[1, 2]
[0, 2, 3]
[0, 1, 3]


Saber que as pessoas são amigas-de-amigas de diversas maneiras parece uma informação interessante, então talvez devo produzir uma contagem de amigos em comum. Definitivamente, devemos usar uma função de ajuda para excluir as pessoas que já são conhecidas do usuário: 

In [39]:
from collections import Counter

def not_the_same(user,other_user): # Verifica se dois usuários são diferentes comparando seus IDs
    return user["id"] != other_user["id"]

def not_friends(user,other_user): # Verifica se 'other_user' não está na lista de amigos de 'user'
    return all(not_the_same(friend,other_user)
               for friend in user["friends"])

def friends_of_friends_ids(user): # Criando um contatodr dos IDs dos amigos dos amigos do usuário, exceto o próprio usuário e seus amigos diretos.
    return Counter(foaf["id"]
                   for friend in user["friends"] # para cada um dos meus amigos
                   for foaf in friend["friends"] # que contam 'their' amigos
                   if not_the_same(user,foaf) # que não sejam eu
                   and not_friends(user,foaf)) # e que não são meus amigos

print (friends_of_friends_ids(users[3]))

Counter({0: 2, 5: 1})
